In [1]:
%matplotlib widget

In [2]:
# Imports
import sys
import os 
import numpy as np
import pandas as pd
from astropy.visualization import hist
from tqdm import tqdm
from scipy import constants

from spaxelsleuth.loaddata.lzifu import load_lzifu_galaxies
from spaxelsleuth.loaddata.sami import load_sami_galaxies
from spaxelsleuth.plotting.plottools import plot_empty_BPT_diagram
from spaxelsleuth.plotting.plottools import vmin_fn, vmax_fn, label_fn, cmap_fn, fname_fn
from spaxelsleuth.plotting.plottools import bpt_colours, bpt_labels, whav_colors, whav_labels
from spaxelsleuth.plotting.plottools import morph_labels, morph_ticks
from spaxelsleuth.plotting.plottools import ncomponents_labels, ncomponents_colours
from spaxelsleuth.plotting.plottools import component_labels, component_colours
from spaxelsleuth.plotting.plotgalaxies import plot2dhistcontours, plot2dscatter, plot2dcontours
from spaxelsleuth.plotting.plot2dmap import plot2dmap
from spaxelsleuth.plotting.sdssimg import plot_sdss_image

import matplotlib
from matplotlib import rc, rcParams
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

from IPython.core.debugger import Tracer

rc("text", usetex=False)
rc("font",**{"family": "serif", "size": 14})
rcParams["savefig.bbox"] = "tight"
rcParams["savefig.format"] = "pdf"
plt.ion()
plt.close("all")


In [3]:
# Options
fig_path = "/priv/meggs3/u5708159/SAMI/figs/paper/"
savefigs = False
bin_type = "default"    # Options: "default" or "adaptive" for Voronoi binning
ncomponents = "recom"   # Options: "1" or "recom"
eline_SNR_min = 5       # Minimum S/N of emission lines to accept
plt.close("all")


In [4]:
# Load the sample
df = load_sami_galaxies(ncomponents=ncomponents,
                        bin_type=bin_type,
                        eline_SNR_min=eline_SNR_min, 
                        vgrad_cut=False,
                        line_amplitude_SNR_cut=True,
                        correct_extinction=False,
                        sigma_gas_SNR_cut=True)

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc,

In [100]:
# How many galaxies are there with inclination < 30 degrees?


array([    289176,     227223,     203148,      49730,      79601,
           278554,     521898,     386402,     185532,     240449,
           492414,     543498,     106343,      63777,     388603,
           203998,     376165,     422761,     381502,     623712,
           229335,     574572,     536363,     323874,      99513,
           230829,     346440,     209664,     323224,      31725,
            93354,     227673,     388424,     535429,     463660,
            22932,     583637,      77445,      36923,     545986,
            84106,     220217,     230774,     100192,      69462,
           402988,     273309,     204836,     623144,     623366,
           534654,      99795,     373248,     144846,     289085,
           323577,      79689,     545925,     184230,     512524,
           418725,      62145,     323504,     220372,      32249,
           203037,     228104,     177297,      99326,     230560,
           345646,     227290,      30914,     543499,     325

In [11]:
df_cut = df.copy()

for ii in [1, 2]:
    df_cut[f"Low flux component (component {ii})"] = False
    cond_low_flux = df_cut[f"HALPHA A (component {ii})"] < 0.05 * df_cut["HALPHA A (component 0)"]
    df_cut.loc[cond_low_flux, f"Low flux component (component {ii})"] = True

    # NaN out rows 
    cols = [f"HALPHA (component {ii})", f"HALPHA error (component {ii})"]
    cols += [f"HALPHA EW (component {ii})", f"HALPHA EW error (component {ii})"]
    cols += [f"v_gas (component {ii})",
             f"sigma_gas (component {ii})",
             f"v_gas error (component {ii})",
             f"sigma_gas error (component {ii})",]
    df_cut.loc[df_cut[f"Low flux component (component {ii})"], cols] = np.nan


In [13]:
# Is the widespread 2nd component in star-forming galaxies due to inclination effects?
# Bar plot showing fraction of multi-component spaxels as a function of inclination
# Compute how many 1, 2, 3 component spaxels there are in bins of SFR surface density (and SFR)
df_SF = df_cut.copy()
df_SF = df_SF[df_SF["BPT (total)"] == "SF"]

for col_x in ["Inclination i (degrees)"]:
    sfr_vals = np.linspace(vmin_fn(col_x), vmax_fn(col_x), 20)
    counts_1 = np.zeros(len(sfr_vals) - 1)
    counts_2 = np.zeros(len(sfr_vals) - 1)
    counts_3 = np.zeros(len(sfr_vals) - 1)
    counts_tot = np.zeros(len(sfr_vals) - 1)

    for ll in range(len(sfr_vals) - 1):
        cond = df_SF[f"{col_x}"] > sfr_vals[ll]
        cond &= df_SF[f"{col_x}"] <= sfr_vals[ll + 1]
        df_subset = df_SF[cond]
        counts_tot[ll] = df_subset.shape[0]
        counts_1[ll] = df_subset[df_subset["Number of components"] == 1].shape[0]
        counts_2[ll] = df_subset[df_subset["Number of components"] == 2].shape[0]
        counts_3[ll] = df_subset[df_subset["Number of components"] == 3].shape[0]

    # Plot
    fig, axs = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(7, 8))
    fig.subplots_adjust(hspace=0)

    # Plot
    axs[0].bar(sfr_vals[:-1], counts_1,
               align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[1],
               label="1 component")
    axs[0].bar(sfr_vals[:-1], counts_2, bottom=counts_1,
               align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[2],
               label="2 components")
    axs[0].bar(sfr_vals[:-1], counts_3, bottom=counts_1 + counts_2,
               align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[3],
               label="3 components")
    axs[0].grid()
    axs[0].set_ylabel(r"$N$")
    axs[0].set_yscale("log")
    axs[0].autoscale(axis="x", enable=True, tight=True)
    axs[0].set_ylim([0.5, None])
    
    axs[1].bar(sfr_vals[:-1], counts_1 / counts_tot  * 100,
           align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[1],
              label="1 component")
    axs[1].bar(sfr_vals[:-1], counts_2 / counts_tot  * 100, bottom=counts_1 / counts_tot * 100,
           align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[2],
              label="2 components")
    axs[1].bar(sfr_vals[:-1], counts_3 / counts_tot * 100, bottom=counts_1 / counts_tot * 100 + counts_2 / counts_tot * 100,
           align="edge", width=np.diff(sfr_vals)[0], color=ncomponents_colours[3],
              label="3 components")
    axs[1].grid()
    axs[1].set_ylabel("Percentage")
    axs[1].autoscale(axis="x", enable=True, tight=True)
    axs[1].autoscale(axis="y", enable=True, tight=True)
    axs[1].set_xlabel(label_fn(col_x))

    axs[1].legend(loc="center left", fontsize="small", bbox_to_anchor=[1.0, 1.0])
        
    if savefigs:
        fname = os.path.join(fig_path, f"hist_SF_only_{col_x.replace(' ', '_')}_ncomponents.pdf")
        print(f"Saving to {fname}")
        fig.savefig(fname, bbox_inches="tight")



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Compare: maps of galaxies before & after flux ratio cut

---
69462
402988
545925

outflow:
623366
418831 (bar)

In [140]:
# Low-inclination galaxies
gals = df.loc[df["Inclination i (degrees)"] <= 30, "ID"].unique()

# Galaxies with a large portion of 2-component spaxels
gals_good = []
for gal in gals:
    df_gal = df[df["ID"] == gal]
    if df_gal[df_gal["Number of components"] > 1].shape[0] > 20:
        gals_good.append(gal)
    

In [141]:
gg = 0

In [162]:
plt.close("all")
gal = gals_good[gg]
df_gal = df[df["ID"] == gal]
df_gal_cut = df_cut[df_cut["ID"] == gal]

fig, axs = plt.subplots(nrows=4, ncols=3, figsize=(13, 17))
fig.subplots_adjust(wspace=0.05, hspace=0.05)

# SDSS image 
ax = plot_sdss_image(df_gal, ax=axs[0][0])
if ax is not None:
    ax.set_title(f"GAMA{gal}")
    lon = ax.coords[0]
    lon.set_ticklabel_visible(False)

# Number of components
_, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
          PA_deg=0,
          col_z="Number of components",
          ax=axs[0][1], 
          plot_colorbar=True, cax=None, cax_orientation="horizontal", 
          show_title=False)
lon = ax.coords[0]
lon.set_ticklabel_visible(False)
lat = ax.coords[1]
lat.set_ticklabel_visible(False)


# BPT classifications 
_, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
          PA_deg=0,
          col_z="BPT (numeric) (total)",
          ax=axs[0][2], 
          plot_colorbar=True, cax=None, cax_orientation="vertical", 
          show_title=False)
lon = ax.coords[0]
lon.set_ticklabel_visible(False)
lat = ax.coords[1]
lat.set_ticklabel_visible(False)

# v_gas
for ii in range(3):
    _, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
              PA_deg=0,
              col_z=f"v_gas (component {ii})",
              ax=axs[1][ii], 
              plot_colorbar=True if ii == 2 else False, cax=None, cax_orientation="vertical", 
              vmin=-200, vmax=+200,
              show_title=False)
    ax.text(s=f"Component {ii + 1}", x=0.05, y=0.95, transform=axs[1][ii].transAxes, verticalalignment="top")
    if ii > 0:
        lat = ax.coords[1]
        lat.set_ticklabel_visible(False)
    lon = ax.coords[0]
    lon.set_ticklabel_visible(False)


# delta sigma 
for ii in range(3):
    _, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
              PA_deg=0,
              col_z=f"sigma_gas (component {ii})",
              ax=axs[2][ii], 
              plot_colorbar=True if ii == 2 else False, cax=None, cax_orientation="vertical", 
              vmin=0, vmax=+200,
              show_title=False)
    ax.text(s=f"Component {ii + 1}", x=0.05, y=0.95, transform=axs[1][ii].transAxes, verticalalignment="top")
    if ii > 0:
        lat = ax.coords[1]
        lat.set_ticklabel_visible(False)
    lon = ax.coords[0]
    lon.set_ticklabel_visible(False)

# EW 
for ii in range(3):
    _, ax = plot2dmap(df_gal=df_gal, bin_type="default", survey="sami",
              PA_deg=0,
              col_z=f"HALPHA EW (component {ii})",
              ax=axs[3][ii], 
              plot_colorbar=True if ii == 2 else False, cax=None, cax_orientation="vertical", 
              show_title=False)
    ax.text(s=f"Component {ii + 1}", x=0.05, y=0.95, transform=axs[2][ii].transAxes, verticalalignment="top")
    if ii > 0:
        lat = ax.coords[1]
        lat.set_ticklabel_visible(False)

gg += 1


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/spaxelsleuth/plotting/plot2dmap.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gal["x, y (pixels)"] = list(zip(df_gal["x (projected, arcsec)"] / as_per_px, df_gal["y (projected, arcsec)"] / as_per_px))


In [97]:
fig, axs = plt.subplots(nrows=4, ncols=3, figsize=(13, 17))
fig.subplots_adjust(wspace=0.05, hspace=0.05)

# SDSS image 
ax = plot_sdss_image(df_gal_cut, ax=axs[0][0])
if ax is not None:
    ax.set_title(f"GAMA{gal}")
    lon = ax.coords[0]
    lon.set_ticklabel_visible(False)

# Number of components
_, ax = plot2dmap(df_gal=df_gal_cut, bin_type="default", survey="sami",
          PA_deg=0,
          col_z="Number of components",
          ax=axs[0][1], 
          plot_colorbar=True, cax=None, cax_orientation="horizontal", 
          show_title=False)
lon = ax.coords[0]
lon.set_ticklabel_visible(False)
lat = ax.coords[1]
lat.set_ticklabel_visible(False)


# BPT classifications 
_, ax = plot2dmap(df_gal=df_gal_cut, bin_type="default", survey="sami",
          PA_deg=0,
          col_z="BPT (numeric) (total)",
          ax=axs[0][2], 
          plot_colorbar=True, cax=None, cax_orientation="vertical", 
          show_title=False)
lon = ax.coords[0]
lon.set_ticklabel_visible(False)
lat = ax.coords[1]
lat.set_ticklabel_visible(False)

# v_gas
for ii in range(3):
    _, ax = plot2dmap(df_gal=df_gal_cut, bin_type="default", survey="sami",
              PA_deg=0,
              col_z=f"v_gas (component {ii})",
              ax=axs[1][ii], 
              plot_colorbar=True if ii == 2 else False, cax=None, cax_orientation="vertical", 
              vmin=-200, vmax=+200,
              show_title=False)
    ax.text(s=f"Component {ii + 1}", x=0.05, y=0.95, transform=axs[1][ii].transAxes, verticalalignment="top")
    if ii > 0:
        lat = ax.coords[1]
        lat.set_ticklabel_visible(False)
    lon = ax.coords[0]
    lon.set_ticklabel_visible(False)


# delta sigma 
for ii in range(3):
    _, ax = plot2dmap(df_gal=df_gal_cut, bin_type="default", survey="sami",
              PA_deg=0,
              col_z=f"sigma_gas (component {ii})",
              ax=axs[2][ii], 
              plot_colorbar=True if ii == 2 else False, cax=None, cax_orientation="vertical", 
              vmin=0, vmax=+200,
              show_title=False)
    ax.text(s=f"Component {ii + 1}", x=0.05, y=0.95, transform=axs[1][ii].transAxes, verticalalignment="top")
    if ii > 0:
        lat = ax.coords[1]
        lat.set_ticklabel_visible(False)
    lon = ax.coords[0]
    lon.set_ticklabel_visible(False)

# EW 
for ii in range(3):
    _, ax = plot2dmap(df_gal=df_gal_cut, bin_type="default", survey="sami",
              PA_deg=0,
              col_z=f"HALPHA EW (component {ii})",
              ax=axs[3][ii], 
              plot_colorbar=True if ii == 2 else False, cax=None, cax_orientation="vertical", 
              show_title=False)
    ax.text(s=f"Component {ii + 1}", x=0.05, y=0.95, transform=axs[2][ii].transAxes, verticalalignment="top")
    if ii > 0:
        lat = ax.coords[1]
        lat.set_ticklabel_visible(False)
        


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/spaxelsleuth/plotting/plot2dmap.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gal["x, y (pixels)"] = list(zip(df_gal["x (projected, arcsec)"] / as_per_px, df_gal["y (projected, arcsec)"] / as_per_px))


In [101]:
gal = 227223
df_gal = df_cut[df_cut["ID"] == gal]
z = df_gal["z_spec"].values[0]

# Load data cube 

# Get wavelength information from header


# Get emission line flux, kinematics from data frame 
x = 25 * 0.5
y = 25 * 0.5

# Generate emission line spectrum based on the best-fit kinematics
df_spaxel = df_gal[(df_gal["x (projected, arcsec)"] == x) & (df_gal["y (projected, arcsec)"] == y)]

# Line amplitude
A_1 = df_spaxel["HALPHA A (component 0)"].values[0]
A_2 = df_spaxel["HALPHA A (component 1)"].values[0]

# Velocity space
v_kms_1 = df_spaxel["v_gas (component 0)"].values[0]
v_kms_2 = df_spaxel["v_gas (component 1)"].values[0]
sigma_kms_1 = df_spaxel["sigma_gas (component 0)"].values[0]
sigma_kms_2 = df_spaxel["sigma_gas (component 1)"].values[0]

# Wavelength space 
lambda_A_1 = df_spaxel["HALPHA lambda_obs (component 0) (Å)"].values[0]
lambda_A_2 = df_spaxel["HALPHA lambda_obs (component 1) (Å)"].values[0]
sigma_A_1 = df_spaxel["HALPHA sigma_gas (component 0) (Å)"].values[0]
sigma_A_2 = df_spaxel["HALPHA sigma_gas (component 1) (Å)"].values[0]


# Plot 
v_vals_kms = np.linspace(-500, 500, 500)
line_1 = A_1 * np.exp( -(v_vals_kms - v_kms_1)**2 / (2 * sigma_kms_1**2))
line_2 = A_2 * np.exp( -(v_vals_kms - v_kms_2)**2 / (2 * sigma_kms_2**2))
line_tot = line_1 + line_2

fig, ax = plt.subplots()
ax.plot(v_vals_kms, line_1)
ax.plot(v_vals_kms, line_2)
ax.plot(v_vals_kms, line_tot)


lambda_vals_A = np.linspace(6562.8 - 50, 6562.8 + 50, 500)
line_1 = A_1 * np.exp( -(lambda_vals_A - lambda_A_1)**2 / (2 * sigma_A_1**2))
line_2 = A_2 * np.exp( -(lambda_vals_A - lambda_A_2)**2 / (2 * sigma_A_2**2))
line_tot = line_1 + line_2

fig, ax = plt.subplots()
ax.plot(lambda_vals_A * (1 + z), line_1)
ax.plot(lambda_vals_A * (1 + z), line_2)
ax.plot(lambda_vals_A * (1 + z), line_tot)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …